In [2]:
import os
import shutil
import torch
import torchvision.models as models
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

In [4]:
# Define paths
train_images_dir = "C:\\Users\\Mohammad\\Desktop\\New folder (2)\\archive-6\\train-pure\\train-pure\\images\\train"
train_labels_dir = "C:\\Users\\Mohammad\\Desktop\\New folder (2)\\archive-6\\train-pure\\train-pure\\labels\\train"
val_images_dir = "C:\\Users\\Mohammad\\Desktop\\New folder (2)\\archive-6\\valid\\valid\\images"
val_labels_dir = "C:\\Users\\Mohammad\\Desktop\\New folder (2)\\archive-6\\valid\\valid\\labels"
test_images_dir = "C:\\Users\\Mohammad\\Desktop\\New folder (2)\\archive-6\\train-pure\\train-pure\\images\\test"
test_labels_dir = "C:\\Users\\Mohammad\\Desktop\\New folder (2)\\archive-6\\train-pure\\train-pure\\labels\\test"

In [6]:
# Define a custom ResNet152 backbone
class CustomResNet152Backbone(torch.nn.Module):
    def __init__(self):
        super(CustomResNet152Backbone, self).__init__()
        self.backbone = models.resnet152(pretrained=True)
        self.backbone = torch.nn.Sequential(*(list(self.backbone.children())[:-2]))

    def forward(self, x):
        return self.backbone(x)

# Load YOLOv8 with a custom backbone
model = YOLO('yolov8s.pt')  # Load a small YOLOv8 model to modify
model.model.backbone = CustomResNet152Backbone()

# Configure the optimizer and loss function
model.model.optimizer = torch.optim.Adam(model.model.parameters(), lr=1e-4)
model.model.loss = torch.nn.BCEWithLogitsLoss()

C:\Users\Mohammad\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Mohammad\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Define paths for train, validation, and test data
data_config = {
    'train': train_images_dir,
    'val': val_images_dir,
    'test': test_images_dir,
    'nc': 1,  # Number of classes
    'names': ['pavement']  # Class names
}

# Save the data config file
import yaml
with open('data.yaml', 'w') as f:
    yaml.dump(data_config, f)

# Train the model
model.train(data='data.yaml', epochs=100, batch=16, imgsz=640, name='yolov8_road_pavement')


Ultralytics YOLOv8.0.196  Python-3.12.4 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8_road_pavement, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscr

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C599035FA0>
fitness: 0.2972084349899648
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.26427])
names: {0: 'pavement'}
plot: True
results_dict: {'metrics/precision(B)': 0.6455129793618928, 'metrics/recall(B)': 0.5588235294117647, 'metrics/mAP50(B)': 0.5936212542218084, 'metrics/mAP50-95(B)': 0.2642736772975377, 'fitness': 0.2972084349899648}
save_dir: WindowsPath('runs/detect/yolov8_road_pavement10')
speed: {'preprocess': 0.2107603815475096, 'inference': 3.5722699148920456, 'loss': 0.0, 'postprocess': 0.8815821221720611}

In [10]:
results=model.val(data='data.yaml', split='test')

Ultralytics YOLOv8.0.196  Python-3.12.4 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\labels\test... 1590 images, 0 ba
val: WARNING  C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\images\test\G0011614_jpg.rf.ccad2055aa6ecc87078ca39ff30513d0.jpg: 1 duplicate labels removed
val: WARNING  C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\images\test\G0011616_jpg.rf.240736dd565d36dff7fb4d9f4aa0c9f2.jpg: 2 duplicate labels removed
val: WARNING  C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\images\test\G0011705_jpg.rf.ce05796e61f41ec5a310fb490ae85297.jpg: 1 duplicate labels removed
val: WARNING  C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\images\test\G0011769_jpg.rf.0a9c7a0febacf2ddcf07f78b3acca0c1.

In [84]:
import os
import cv2
import shutil
import random
import matplotlib.pyplot as plt

# Function to draw bounding boxes from labels with custom text
def draw_bounding_boxes_from_labels(image_path, label_path, class_names, output_dir, custom_text='Damage'):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    with open(label_path, 'r') as f:
        labels = f.readlines()

    for label in labels:
        parts = label.strip().split()
        class_id = int(parts[0])
        x_center, y_center, box_width, box_height = map(float, parts[1:])

        x1 = int((x_center - box_width / 2) * w)
        y1 = int((y_center - box_height / 2) * h)
        x2 = int((x_center + box_width / 2) * w)
        y2 = int((y_center + box_height / 2) * h)

        color = (255, 0, 0)  # Red color for original bounding box
        label_text = custom_text  # Custom label text
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Save the image with bounding boxes
    os.makedirs(output_dir, exist_ok=True)
    base_name = os.path.basename(image_path)
    file_name, file_ext = os.path.splitext(base_name)
    output_path = os.path.join(output_dir, file_name + '_orig' + file_ext)
    cv2.imwrite(output_path, image)
    return output_path

# Define a function to plot predicted bounding boxes on images with custom text
def plot_predicted_bounding_boxes(image_path, boxes, scores, labels, class_names, output_dir, conf_threshold=0.5, custom_text='Damage'):
    image = cv2.imread(image_path)
    for box, score, label in zip(boxes, scores, labels):
        if score >= conf_threshold:
            x1, y1, x2, y2 = map(int, box)
            color = (0, 255, 0)  # Green color for predicted bounding box
            label_text = f"{custom_text}: {score:.2f}"  # Custom label text
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Save the image with bounding boxes
    os.makedirs(output_dir, exist_ok=True)
    base_name = os.path.basename(image_path)
    file_name, file_ext = os.path.splitext(base_name)
    output_path = os.path.join(output_dir, file_name + '_pred' + file_ext)
    cv2.imwrite(output_path, image)
    return output_path

# Get all test image paths
test_image_paths = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

# Randomly select 20 images from the test set
num_images_to_visualize = 100
random_image_paths = random.sample(test_image_paths, num_images_to_visualize)

# Run inference on the selected test images and visualize results
predictions = []
for img_path in random_image_paths:
    results = model(img_path)
    predictions.append((img_path, results[0].boxes.xyxy.cpu().numpy(), results[0].boxes.conf.cpu().numpy(), results[0].boxes.cls.cpu().numpy()))

# Visualize and save predictions
output_dir = "D:\\YOLO\\New folder"
saved_image_paths = []
for img_path, boxes, scores, labels in predictions:
    # Get the corresponding label file path
    label_path = img_path.replace('images', 'labels').replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt')

    # Draw bounding boxes from labels on the original image and save it
    original_output_path = draw_bounding_boxes_from_labels(img_path, label_path, class_names=data_config['names'], output_dir=output_dir, custom_text='Damage')
    
    # Plot predicted bounding boxes and save the image with predicted bounding boxes
    saved_image_path = plot_predicted_bounding_boxes(img_path, boxes, scores, labels, class_names=data_config['names'], output_dir=output_dir, custom_text='Damage')
    saved_image_paths.append((original_output_path, saved_image_path))

# Display the saved images
for original_path, saved_image_path in saved_image_paths:
    # Display original image with ground truth bounding boxes
    original_img = cv2.imread(original_path)
    plt.figure(figsize=(10, 10))
    plt.title("Original Image with Ground Truth Bounding Boxes")
    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    
    # Display image with predicted bounding boxes
    pred_img = cv2.imread(saved_image_path)
    plt.figure(figsize=(10, 10))
    plt.title("Image with Predicted Bounding Boxes")
    plt.imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()



image 1/1 C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\images\test\India_005399_jpg.rf.808b173d4f6a413ad7d790e240663244.jpg: 640x640 (no detections), 60.8ms
Speed: 4.4ms preprocess, 60.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\images\test\India_008271_jpg.rf.b968924a8d8a12574497cc2a91ffb362.jpg: 640x640 1 pavement, 59.9ms
Speed: 4.0ms preprocess, 59.9ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\images\test\India_002880_jpg.rf.81a481f4b7c4c7a7f4cdae9003dd4f5e.jpg: 640x640 1 pavement, 54.6ms
Speed: 4.0ms preprocess, 54.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\images\test\India_006640_jpg.rf.2d53bcf9c94d9024d764f52e22ba4240.jpg: 640x640

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

...

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>